# 버거지수

"한 도시의 발전 수준은 (버거킹의 개수+맥도날드의 개수+KFC의 개수)/롯데리아의 개수를 계산하여 높게 나올수록 더 발전된 도시라고 할 수 있다"

In [1]:
import sys
import numpy as np
import pandas as pd
import time
import json
import re

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm

# Statistical data visualization using matplotlib.
import seaborn as sns

# Python Data. Leaflet.js Maps.
import folium

# matplotlib: plotting with Python.
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager

# Set the matplotlib color cycle using a seaborn palette.
sns.set_palette('pastel')

# plot outputs appear and be stored within the notebook.
%matplotlib inline

# A module for finding, managing, and using fonts across platforms.
mpl.font_manager._rebuild()
sorted([f.name for f in mpl.font_manager.fontManager.ttflist if f.name.startswith('Malgun')])

# 폰트 설정
mpl.rc('font', family='Malgun Gothic')

# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

# An importlib package installable from any python version
import importlib
import modules.fs as fs

## 통계청: 행정구역(시군구)별, 성별 인구수

In [2]:
population = fs.read_csv('./data/population_and_navi_202010.csv')

population['SIG_CD'] = population['시군구코드']
population['SIG_X'] = population['시군구_x']
population['SIG_Y'] = population['시군구_y']

population.head(1)

,시도명,시도코드,시군구명,시군구코드,인구수,남자수,여자수,시군구_x,시군구_y,SIG_CD,SIG_X,SIG_Y
0,서울특별시,11,종로구,11110,151290,73746,77544,954859.306142,1.953755e+06,11110,954859.306142,1.953755e+06


## 공공데이터포털: 상가(상권)정보

### 패스트푸드 (맥도날드 & KFC & 버거킹 & 롯데리아)

In [3]:
store = fs.read_csv('./data/macdonald_in_korea_2020.csv', sep='|')
store = store[['시군구코드', '상호명', '지점명', '도로명주소', '경도', '위도']]
store['프랜차이즈'] = '맥도날드'
store_count = store.groupby('시군구코드')['상호명'].count()
store_count.fillna(0, inplace=True)

macdonald = pd.merge(store, store_count, on='시군구코드')
macdonald.rename(columns={'상호명_x':'상호명', '상호명_y':'시군구점포수'}, inplace=True)
macdonald = pd.merge(population, macdonald, on='시군구코드', how='left')
macdonald.fillna(0, inplace=True)
macdonald.head(1)

,시도명,시도코드,시군구명,시군구코드,인구수,남자수,여자수,시군구_x,시군구_y,SIG_CD,SIG_X,SIG_Y,상호명,지점명,도로명주소,경도,위도,프랜차이즈,시군구점포수
0,서울특별시,11,종로구,11110,151290,73746,77544,954859.306142,1.953755e+06,11110,954859.306142,1.953755e+06,0,0,0,0.0,0.0,0,0.0


In [4]:
store = fs.read_csv('./data/burgerking_in_korea_2020.csv', sep='|')
store = store[['시군구코드', '상호명', '지점명', '도로명주소', '경도', '위도']]
store_count = store.groupby('시군구코드')['상호명'].count()
store_count.fillna(0, inplace=True)
store['프랜차이즈'] = '버거킹'

burgerking = pd.merge(store, store_count, on='시군구코드')
burgerking.rename(columns={'상호명_x':'상호명', '상호명_y':'시군구점포수'}, inplace=True)
burgerking = pd.merge(population, burgerking, on='시군구코드', how='left')
burgerking.fillna(0, inplace=True)
burgerking.head(1)

,시도명,시도코드,시군구명,시군구코드,인구수,남자수,여자수,시군구_x,시군구_y,SIG_CD,SIG_X,SIG_Y,상호명,지점명,도로명주소,경도,위도,프랜차이즈,시군구점포수
0,서울특별시,11,종로구,11110,151290,73746,77544,954859.306142,1.953755e+06,11110,954859.306142,1.953755e+06,0,0,0,0.0,0.0,0,0.0


In [10]:
store = fs.read_csv('./data/kfc_in_korea_2020.csv', sep='|')
store = store[['시군구코드', '상호명', '지점명', '도로명주소', '경도', '위도']]
store_count = store.groupby('시군구코드')['상호명'].count()
store_count.fillna(0, inplace=True)
store['프랜차이즈'] = 'kfc'

kfc = pd.merge(store, store_count, on='시군구코드')
kfc.rename(columns={'상호명_x':'상호명', '상호명_y':'시군구점포수'}, inplace=True)
kfc = pd.merge(population, kfc, on='시군구코드', how='left')
kfc.fillna(0, inplace=True)
kfc.head(1)

,시도명,시도코드,시군구명,시군구코드,인구수,남자수,여자수,시군구_x,시군구_y,SIG_CD,SIG_X,SIG_Y,버거지수,상호명,지점명,도로명주소,경도,위도,프랜차이즈,시군구점포수
0,서울특별시,11,종로구,11110,151290,73746,77544,954859.306142,1.953755e+06,11110,954859.306142,1.953755e+06,0.0,0,0,0,0.0,0.0,0,0.0


In [6]:
store = fs.read_csv('./data/lotteria_in_korea_2020.csv', sep='|')
store = store[['시군구코드', '상호명', '지점명', '도로명주소', '경도', '위도']]
store_count = store.groupby('시군구코드')['상호명'].count()
store_count.fillna(0, inplace=True)
store['프랜차이즈'] = '롯데리아'

lotteria = pd.merge(store, store_count, on='시군구코드')
lotteria.rename(columns={'상호명_x':'상호명', '상호명_y':'시군구점포수'}, inplace=True)
lotteria = pd.merge(population, lotteria, on='시군구코드', how='left')
lotteria.fillna(0, inplace=True)
lotteria.head(1)

,시도명,시도코드,시군구명,시군구코드,인구수,남자수,여자수,시군구_x,시군구_y,SIG_CD,SIG_X,SIG_Y,상호명,지점명,도로명주소,경도,위도,프랜차이즈,시군구점포수
0,서울특별시,11,종로구,11110,151290,73746,77544,954859.306142,1.953755e+06,11110,954859.306142,1.953755e+06,0,0,0,0.0,0.0,0,0.0


## 버거지수 = (버거킹의 개수 + 맥도날드의 개수 + KFC의 개수) / 롯데리아의 개수

In [15]:
df = population
df['버거지수'] = (macdonald['시군구점포수'] + burgerking['시군구점포수'] + kfc['시군구점포수']) / lotteria['시군구점포수']
df.fillna(0, inplace=True)
df = df[['시도명', '시군구명', '인구수', '남자수', '여자수', '버거지수', 'SIG_CD', 'SIG_X', 'SIG_Y']]
df

,시도명,시군구명,인구수,남자수,여자수,버거지수,SIG_CD,SIG_X,SIG_Y
0,서울특별시,종로구,151290,73746,77544,0.000000,11110,9.548593e+05,1.953755e+06
1,서울특별시,중구,126171,61910,64261,0.000000,11140,9.559310e+05,1.951526e+06
2,서울특별시,용산구,228670,110356,118314,0.000000,11170,9.541220e+05,1.948855e+06
3,서울특별시,성동구,300889,147273,153616,0.000000,11200,9.595713e+05,1.950601e+06
4,서울특별시,광진구,351350,170262,181088,0.000000,11215,9.630325e+05,1.949996e+06
...,...,...,...,...,...,...,...,...,...
245,경상남도,함양군,39637,19097,20540,1.777778,48870,1.021646e+06,1.728149e+06
246,경상남도,거창군,62179,30345,31834,1.777778,48880,1.037957e+06,1.746249e+06
247,경상남도,합천군,45204,21655,23549,1.555556,48890,1.060001e+06,1.730364e+06
248,제주특별자치도,제주시,489405,245533,243872,1.555556,50110,9.064102e+05,1.498856e+06


## 지도

In [6]:
import modules.plot as plot
importlib.reload(plot)

plot.choropleth(df, column='버거지수', figsize=(12, 12))
# plot.choropleth(df, column='SIG_CD', figsize=(12, 12))

NameError: name 'df' is not defined